In [1]:
from twinyn.agents.prompts import *

import os
import psycopg2
from pathlib import Path
from dotenv import load_dotenv
from autogen import ConversableAgent, AssistantAgent
from autogen.coding import LocalCommandLineCodeExecutor
from autogen.coding.func_with_reqs import with_requirements
load_dotenv()

work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

In [2]:
import agentops
agentops.init(os.getenv("AGENTOPS_API_KEY"))

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=1d68f9eb-8463-411f-b350-2e8035242241


In [3]:
@with_requirements(python_packages=["psycopg2"], global_imports=["psycopg2", "os"])
def execute_sql(query: str) -> list:
    """Execute SQL statement and return a list of results. Does NOT print the results."""
    conn = psycopg2.connect(os.getenv("CONNECTION_URL"))
    with conn.cursor() as cursor:
        cursor.execute(
            query
        )
        res = cursor.fetchall()
        return res

In [4]:

# TODO:
# - include multiple seed prompts
# - aggregate and manage results, analysis, and instructions from all the seed prompts
# - 2nd order analysis from these resultant artifacts
# - a nice API also maybe?

executor = LocalCommandLineCodeExecutor(work_dir=work_dir, functions=[execute_sql])
seed_prompts = [
    "Peak traffic time in the past 5 hours bucketed in 30 minute intervals",
    "Requests by country in the past 5 hours"
]

SQL_AGENT_SYSTEM_PROMPT += executor.format_functions_for_prompt()

# SQL agent who's sole purpose is to write SQL queries and write python code to execute it (using the `execute_sql` function).
sql_agent = ConversableAgent(
    name="sql_agent",
    system_message=SQL_AGENT_SYSTEM_PROMPT,
    # set "cache_seed" to None to NOT use cached responses to same queries
    llm_config={"config_list": [{"model": "gpt-4o-mini", "api_key": os.getenv("OPENAI_API_KEY"), "price": [0.00250, 0.01]}]},
    code_execution_config=False,
    human_input_mode="NEVER"
)

# a code executor agent, as the name indicates. no LLM is configured for this agent.
code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"), # is not called further when TERMINATE is recieved
    llm_config=False,
    code_execution_config={
        "executor": executor,
    },
    human_input_mode="NEVER",
)

# an analyst agent that gives "Analysis" and "Further Instructions" based on the output from the `sql_agent`'s query output.
analyst_agent = AssistantAgent(
    name="analyst_agent",
    system_message=ANALYST_AGENT_SYSTEM_PROMPT,
    llm_config={"config_list": [{"model": "gpt-4o-mini", "api_key": os.getenv("OPENAI_API_KEY"), "price": [0.000150, 0.000600]}]},
)

In [5]:

# TODO:
# - evaluate if this functions works as expected. expectation: take both the execution output (given by `code_executor_agent`)
#   and the response to the output (which also contains TERMINATE) by `sql_agent` as carryover to the `analyst_agent`
def custom_message(sender: ConversableAgent, recipient: ConversableAgent, context: dict) -> str | dict:
    carryover = context.get("carryover", "")
    if isinstance(carryover, list):
        carryover = '\n'.join(carryover[-3:])
    final_msg = "What do you think of the results? Do you find any peculiarity or anything that require further querying?" + "\nContext: \n" + carryover
    return final_msg

In [6]:

# kicks off execution with the `code_executor_agent` calling `sql_agent` with the seed_prompt
# more info: https://microsoft.github.io/autogen/docs/tutorial/conversation-patterns#sequential-chats
# api-ref: https://microsoft.github.io/autogen/docs/reference/agentchat/conversable_agent#initiate_chats and #initiate_chat
chat_results = []
for seed_prompt in seed_prompts[:1]:
    chat_results.append(
        code_executor_agent.initiate_chats(
            [
                {
                    "recipient": sql_agent,
                    "message": seed_prompt,
                    "clear_history": True,
                    "silent": False,
                    "max_turns": None, # indicates to wait till "TERMINATE" is sent by this agent
                    
                    "summary_method": "last_msg"
                },
                {
                    "recipient": analyst_agent,
                    "message": custom_message,
                    "max_turns": 1,
                    "summary_method": "last_msg",
                }
            ]
        )
    )


********************************************************************************
Starting a new chat....

********************************************************************************
code_executor_agent (to sql_agent):

Peak traffic time in the past 5 hours bucketed in 30 minute intervals

--------------------------------------------------------------------------------
sql_agent (to code_executor_agent):

To identify the peak traffic times in the past 5 hours, we can aggregate the data from the server_log table by counting the number of requests in 30-minute intervals. We'll also need to consider the last request time to define the starting point for our 5-hour period.

Here’s the plan:

1. Retrieve the last request time from the server_log to set the reference for our 5-hour window.
2. Generate a series of 30-minute intervals within that 5-hour window.
3. Count the requests in each interval, grouping them appropriately.
4. Sort the results to find the intervals with the highest n

Note: gpt-4o works the best for SQL queries. mini is shit

In [7]:
agentops.end_session("Success")

🖇 AgentOps: This run's cost $0.000443
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=1d68f9eb-8463-411f-b350-2e8035242241


In [46]:
chat_results[0][0].chat_history[5]

{'content': 'The peak traffic time in the past 5 hours, bucketed in 30-minute intervals, was on January 22, 2019, at 11:01 AM with a total of 3815 requests.\n\nTERMINATE',
 'role': 'user'}